In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search
)

In [3]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-07-15 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-07-15 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-07-15 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-07-15 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-07-15 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(13551, 384)

# Baseline: Averaging history and use FFN

Here, we average the full history of a path and concatenate it the current embedding (the total number of features that are passed into the FFN is `2 * sbert_embeddings.shape[0]`).

Here, we will run the hyperparameter search to implement the FFN with the same parameters as the standard FFN baseline on the sentence embeddings. Going to try out some variations (1 hidden layer, 2 hidden layers and 3 hidden layers - all of size 100).

In [7]:
num_epochs = 100
hidden_dim_sizes = [[128,128],[256,256],[512,512]]
dropout_rates = [0.5, 0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [8]:
ffn_mean_history, best_ffn_mean_history, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_mean_history_focal_{gamma}.csv",
    verbose=False
)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the mean history for each item in the dataframe


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the mean history for each item in the dataframe


  0%|          | 0/13551 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_mean_history_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_mean_history_focal_2_best_model.csv


In [9]:
ffn_mean_history

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size,model_id,input_dim
0,0.620287,0.650265,0.565030,"[0.7478684531059684, 0.515850144092219, 0.4313...",0.557836,"[0.7881899871630296, 0.4649350649350649, 0.420...",0.577904,"[0.7114716106604867, 0.5792880258899676, 0.442...",0.888382,0.577251,...,0.5,0.0010,0,focal,2,False,None,64,0.00,768
0,0.599939,0.661620,0.556837,"[0.762632197414806, 0.5201238390092878, 0.3877...",0.555053,"[0.7735399284862932, 0.49851632047477745, 0.39...",0.559422,"[0.7520278099652375, 0.5436893203883495, 0.382...",0.757981,0.582938,...,0.5,0.0010,1,focal,2,False,None,64,0.00,768
0,0.590228,0.666162,0.556701,"[0.7745732783990582, 0.5235772357723577, 0.371...",0.551336,"[0.7870813397129187, 0.5261437908496732, 0.340...",0.564296,"[0.7624565469293163, 0.5210355987055016, 0.409...",0.734781,0.599052,...,0.5,0.0010,12,focal,2,False,None,64,0.00,768
0,0.598081,0.642695,0.544994,"[0.7504553734061931, 0.5106382978723404, 0.373...",0.534916,"[0.7882653061224489, 0.4813753581661891, 0.335...",0.560872,"[0.7161066048667439, 0.5436893203883495, 0.422...",0.706463,0.581043,...,0.5,0.0010,123,focal,2,False,None,64,0.00,768
0,0.587664,0.642695,0.550627,"[0.7467968273337401, 0.5111441307578009, 0.393...",0.540101,"[0.788659793814433, 0.4725274725274725, 0.3591...",0.567343,"[0.709154113557358, 0.5566343042071198, 0.4362...",0.735837,0.576303,...,0.5,0.0010,1234,focal,2,False,None,64,0.00,768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.597232,0.626798,0.533574,"[0.729426433915212, 0.510752688172043, 0.36054...",0.530591,"[0.7894736842105263, 0.4367816091954023, 0.365...",0.549486,"[0.6778679026651216, 0.6148867313915858, 0.355...",0.726641,0.576303,...,0.1,0.0005,0,focal,2,False,None,64,0.26,768
0,0.604120,0.612415,0.526421,"[0.7144670050761421, 0.4993464052287582, 0.365...",0.523441,"[0.7896213183730715, 0.41885964912280704, 0.36...",0.546542,"[0.6523754345307068, 0.6181229773462783, 0.369...",0.728273,0.573460,...,0.1,0.0005,1,focal,2,False,None,64,0.26,768
0,0.596676,0.669190,0.545200,"[0.7764156450671338, 0.5271084337349399, 0.332...",0.551540,"[0.7823529411764706, 0.49295774647887325, 0.37...",0.544071,"[0.7705677867902665, 0.5663430420711975, 0.295...",0.742722,0.615166,...,0.1,0.0005,12,focal,2,False,None,64,0.26,768
0,0.593227,0.664648,0.552016,"[0.7705882352941177, 0.5205479452054794, 0.364...",0.552096,"[0.7825567502986858, 0.49137931034482757, 0.38...",0.553791,"[0.7589803012746235, 0.5533980582524272, 0.348...",0.735777,0.624645,...,0.1,0.0005,123,focal,2,False,None,64,0.26,768


In [10]:
ffn_mean_history.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_89745/1445575783.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_mean_history.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


loss  accuracy        f1  \
hidden_dim dropout_rate learning_rate                                 
(128, 128) 0.1          0.0001         0.603667  0.638456  0.536233   
                        0.0005         0.597897  0.650114  0.545722   
                        0.0010         0.601654  0.647086  0.546097   
           0.2          0.0001         0.611409  0.644058  0.535514   
                        0.0005         0.601698  0.649357  0.547351   
                        0.0010         0.596159  0.656018  0.550280   
           0.5          0.0001         0.623353  0.648145  0.532049   
                        0.0005         0.611580  0.632703  0.537305   
                        0.0010         0.599240  0.652687  0.554838   
(256, 256) 0.1          0.0001         0.604406  0.638456  0.533065   
                        0.0005         0.605824  0.626949  0.534716   
                        0.0010         0.598170  0.633914  0.542899   
           0.2          0.0001         0.604062  0.645269  0.537198   
                        0.0005         0.610383  0.614837  0.529144   
                        0.0010         0.598330  0.626949  0.537192   
           0.5          0.0001         0.604377  0.637245  0.536313   
                        0.0005         0.601289  0.648145  0.542643   
                        0.0010         0.598950  0.641484  0.544581   
(512, 512) 0.1          0.0001         0.603431  0.647237  0.531310   
                        0.0005         0.596814  0.645269  0.541536   
                        0.0010         0.598130  0.642846  0.534385   
           0.2          0.0001         0.600687  0.642998  0.535052   
                        0.0005         0.597035  0.645420  0.541072   
                        0.0010         0.598065  0.642241  0.536995   
           0.5          0.0001         0.601563  0.638002  0.529034   
                        0.0005         0.601504  0.637396  0.539162   
                        0.0010         0.599150  0.641332  0.539799   

                                       precision    recall  valid_loss  \
hidden_dim dropout_rate learning_rate                                    
(128, 128) 0.1          0.0001          0.530660  0.545523    0.735810   
                        0.0005          0.540936  0.554622    0.735850   
                        0.0010          0.543250  0.555837    0.753466   
           0.2          0.0001          0.531167  0.542418    0.732298   
                        0.0005          0.541712  0.557751    0.745063   
                        0.0010          0.546397  0.558088    0.746750   
           0.5          0.0001          0.534394  0.539644    0.737189   
                        0.0005          0.532849  0.550530    0.749629   
                        0.0010          0.547848  0.565968    0.764689   
(256, 256) 0.1          0.0001          0.529126  0.540338    0.742376   
                        0.0005          0.527901  0.552873    0.741953   
                        0.0010          0.537255  0.562470    0.726800   
           0.2          0.0001          0.534008  0.542893    0.741452   
                        0.0005          0.522432  0.553175    0.735926   
                        0.0010          0.531044  0.558184    0.726059   
           0.5          0.0001          0.531616  0.545499    0.736308   
                        0.0005          0.540663  0.553847    0.740034   
                        0.0010          0.536415  0.559649    0.744373   
(512, 512) 0.1          0.0001          0.533679  0.533434    0.735374   
                        0.0005          0.540717  0.550183    0.735317   
                        0.0010          0.532553  0.545666    0.747209   
           0.2          0.0001          0.531410  0.541686    0.735515   
                        0.0005          0.539769  0.550905    0.733104   
                        0.0010          0.533760  0.549801    0.742842   
           0.5          0.0001          0.527561  0.534644 

In [11]:
best_ffn_mean_history

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size,input_dim
0,0.614325,0.632854,0.496199,"[0.7486631016042781, 0.4888888888888889, 0.251...",0.509950,"[0.7682926829268293, 0.4282238442822384, 0.333...",0.500311,"[0.7300115874855156, 0.56957928802589, 0.20134...",0.757650,0.596209,...,"(512, 512)",0.2,0.001,0,focal,2,False,None,64,768
0,0.594484,0.604845,0.518151,"[0.7103274559193955, 0.4915254237288135, 0.352...",0.507889,"[0.7779310344827586, 0.43609022556390975, 0.30...",0.542012,"[0.6535341830822712, 0.5631067961165048, 0.409...",0.694741,0.581043,...,"(512, 512)",0.2,0.001,1,focal,2,False,None,64,768
0,0.593015,0.676003,0.560954,"[0.7800925925925926, 0.529886914378029, 0.3728...",0.561645,"[0.7791907514450868, 0.5290322580645161, 0.376...",0.560289,"[0.7809965237543454, 0.5307443365695793, 0.369...",0.766219,0.623697,...,"(512, 512)",0.2,0.001,12,focal,2,False,None,64,768
0,0.590471,0.659349,0.556517,"[0.7637462235649547, 0.5368731563421828, 0.368...",0.549152,"[0.797979797979798, 0.4932249322493225, 0.35625]",0.567959,"[0.7323290845886443, 0.5889967637540453, 0.382...",0.739479,0.613270,...,"(512, 512)",0.2,0.001,123,focal,2,False,None,64,768
0,0.598030,0.638153,0.553156,"[0.7405102675793405, 0.5212298682284041, 0.397...",0.540165,"[0.7997311827956989, 0.47593582887700536, 0.34...",0.578435,"[0.6894553881807648, 0.5760517799352751, 0.469...",0.756121,0.566825,...,"(512, 512)",0.2,0.001,1234,focal,2,False,None,64,768


In [12]:
best_ffn_mean_history["f1"].mean()

0.5369954494330182

In [13]:
best_ffn_mean_history["precision"].mean()

0.5337600304683519

In [14]:
best_ffn_mean_history["recall"].mean()

0.5498013664395671

In [15]:
np.stack(best_ffn_mean_history["f1_scores"]).mean(axis=0)

array([0.74866793, 0.51368085, 0.34863757])

In [16]:
np.stack(best_ffn_mean_history["precision_scores"]).mean(axis=0)

array([0.78462509, 0.47250142, 0.34415358])

In [17]:
np.stack(best_ffn_mean_history["recall_scores"]).mean(axis=0)

array([0.71726535, 0.56569579, 0.36644295])

## KFold

We can repeat this but use K-Fold evaluation instead - by default, we have $K=5$ folds.

In [18]:
ffn_mean_history_kfold, best_ffn_mean_history_kfold, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_mean_history_focal_{gamma}_kfold.csv",
    verbose=False
)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the mean history for each item in the dataframe


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the mean history for each item in the dataframe


  0%|          | 0/13551 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_mean_history_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_mean_history_focal_2_kfold_best_model.csv


In [19]:
ffn_mean_history_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size,model_id,input_dim
0,None,0.609053,0.512510,"[0.7361441261103465, 0.4519480519480519, 0.349...",0.508812,"[0.764950598023921, 0.4750853242320819, 0.2864]",0.529483,"[0.7094285025319508, 0.4309597523219814, 0.448...",None,0.698377,...,0.5,0.0010,0,focal,2,True,5,64,0.00,768
0,None,0.615760,0.523320,"[0.7398394380331158, 0.4565916398713827, 0.373...",0.517636,"[0.7709803921568628, 0.47491638795986624, 0.30...",0.542530,"[0.7111164697371594, 0.43962848297213625, 0.47...",None,0.697370,...,0.5,0.0010,1,focal,2,True,5,64,0.00,768
0,None,0.608901,0.523123,"[0.7312348668280871, 0.45924516333650495, 0.37...",0.516667,"[0.7754054054054054, 0.47074122236671, 0.30385...",0.547751,"[0.691825415963347, 0.448297213622291, 0.50312...",None,0.695915,...,0.5,0.0010,12,focal,2,True,5,64,0.00,768
0,None,0.608444,0.519571,"[0.7333248634226909, 0.45434439178515, 0.37104...",0.513181,"[0.7749731471535983, 0.4638709677419355, 0.300...",0.541827,"[0.6959247648902821, 0.4452012383900929, 0.484...",None,0.696586,...,0.5,0.0010,123,focal,2,True,5,64,0.00,768
0,None,0.614388,0.522307,"[0.7384422110552764, 0.4604878048780488, 0.367...",0.517951,"[0.7707841594544977, 0.4849315068493151, 0.298...",0.542565,"[0.7087050880154329, 0.43839009287925695, 0.48...",None,0.700168,...,0.5,0.0010,1234,focal,2,True,5,64,0.00,768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.615150,0.526387,"[0.7350557244174266, 0.47358607830950905, 0.37...",0.519041,"[0.7740730861563083, 0.47535870243293826, 0.30...",0.545731,"[0.6997829756450447, 0.4718266253869969, 0.465...",None,0.700951,...,0.1,0.0005,0,focal,2,True,5,64,0.26,768
0,None,0.616522,0.526570,"[0.7326908445346184, 0.48177311169437154, 0.36...",0.518465,"[0.7726664883658733, 0.4553472987872106, 0.327...",0.540373,"[0.6966481794068001, 0.5114551083591331, 0.413...",None,0.695020,...,0.1,0.0005,1,focal,2,True,5,64,0.26,768
0,None,0.599604,0.518161,"[0.7195106078354808, 0.4701006512729426, 0.364...",0.509994,"[0.7816742081447964, 0.450368689733409, 0.2979...",0.542912,"[0.6665059078852182, 0.491640866873065, 0.4705...",None,0.683828,...,0.1,0.0005,12,focal,2,True,5,64,0.26,768
0,None,0.618046,0.525479,"[0.7390265375424474, 0.4660428614548747, 0.371...",0.517592,"[0.7723449001051524, 0.4546525323910483, 0.325...",0.539424,"[0.7084639498432602, 0.4780185758513932, 0.431...",None,0.704645,...,0.1,0.0005,123,focal,2,True,5,64,0.26,768


In [20]:
ffn_mean_history_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_89745/2776369505.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_mean_history_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
(128, 128) 0.1          0.0001         0.619875  0.530519   0.522490   
                        0.0005         0.604237  0.517631   0.510398   
                        0.0010         0.600213  0.518671   0.511334   
           0.2          0.0001         0.620790  0.532859   0.524674   
                        0.0005         0.606584  0.518634   0.511179   
                        0.0010         0.604694  0.520175   0.513360   
           0.5          0.0001         0.621856  0.534131   0.525818   
                        0.0005         0.615943  0.527436   0.521376   
                        0.0010         0.611309  0.520166   0.514849   
(256, 256) 0.1          0.0001         0.619875  0.529476   0.521667   
                        0.0005         0.608596  0.521280   0.514675   
                        0.0010         0.608871  0.518611   0.514164   
           0.2          0.0001         0.619082  0.527462   0.519990   
                        0.0005         0.608718  0.522256   0.514982   
                        0.0010         0.610639  0.521738   0.517854   
           0.5          0.0001         0.622283  0.533289   0.525293   
                        0.0005         0.612376  0.526291   0.518159   
                        0.0010         0.613992  0.527709   0.521137   
(512, 512) 0.1          0.0001         0.617802  0.525855   0.518628   
                        0.0005         0.613931  0.525471   0.517472   
                        0.0010         0.612559  0.524053   0.518125   
           0.2          0.0001         0.618077  0.525102   0.517811   
                        0.0005         0.612346  0.526782   0.517961   
                        0.0010         0.611980  0.523597   0.517709   
           0.5          0.0001         0.619997  0.527187   0.519755   
                        0.0005         0.616827  0.525865   0.519799   
                        0.0010         0.613169  0.529932   0.521594   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
(128, 128) 0.1          0.0001         0.546438        0.694997  0.628785   
                        0.0005         0.539416        0.691192  0.632093   
                        0.0010         0.544837        0.688103  0.628756   
           0.2          0.0001         0.550204        0.693587  0.627973   
                        0.0005         0.538435        0.693587  0.633583   
                        0.0010         0.545012        0.690274  0.630475   
           0.5          0.0001         0.552570        0.694729  0.628422   
                        0.0005         0.548514        0.696609  0.632902   
                        0.0010         0.540831        0.697683  0.635376   
(256, 256) 0.1          0.0001         0.544557        0.697124  0.631280   
                        0.0005         0.543589        0.694035  0.633598   
                        0.0010         0.540041        0.693632  0.630442   
           0.2          0.0001         0.542238        0.698243  0.633342   
                        0.0005         0.544230        0.694191  0.634166   
                        0.0010         0.545142        0.695758  0.632418   
           0.5          0.0001         0.549877        0.696564  0.631745   
                        0.0005         0.547088        0.696363  0.635774   
                        0.0010         0.550985        0.695132  0.633212   
(512, 512) 0.1          0.0001         0.540843        0.701466  0.638275   
                        0.0005         0.542831        0.696698  0.635365   
                        0.0010         0.545618        0.696184  0.632299   
           0.2          0.0001         0.538550        0.701556  0.637588   
                        0.0005         0.545969        0.694661  0.634936   
                        0

In [21]:
best_ffn_mean_history_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size,input_dim
0,None,0.614693,0.522507,"[0.7347194388777555, 0.46078129806213475, 0.37...",0.514945,"[0.7643992702632265, 0.4578239608801956, 0.322...",0.536778,"[0.7072582589823969, 0.46377708978328175, 0.43...",None,0.701735,...,"(512, 512)",0.1,0.0001,0,focal,2,True,5,64,768
0,None,0.622314,0.526953,"[0.7424392662369855, 0.46440999683644413, 0.37...",0.520731,"[0.7638357561846467, 0.47477360931435963, 0.32...",0.539917,"[0.7222088256571015, 0.4544891640866873, 0.443...",None,0.702182,...,"(512, 512)",0.1,0.0001,1,focal,2,True,5,64,768
0,None,0.616217,0.521673,"[0.7360279441117764, 0.4668946223189307, 0.362...",0.515007,"[0.7624709227190488, 0.46878901373283394, 0.31...",0.534803,"[0.711357607909332, 0.465015479876161, 0.42803...",None,0.703861,...,"(512, 512)",0.1,0.0001,12,focal,2,True,5,64,768
0,None,0.615912,0.529236,"[0.7320327249842667, 0.4692667706708268, 0.386...",0.521143,"[0.765666140073723, 0.4729559748427673, 0.3248...",0.547904,"[0.7012298046780805, 0.46563467492260063, 0.47...",None,0.698601,...,"(512, 512)",0.1,0.0001,123,focal,2,True,5,64,768
0,None,0.619875,0.528905,"[0.7388854101440201, 0.4677670507630022, 0.380...",0.521312,"[0.7686294945284002, 0.4705513784461153, 0.324...",0.544815,"[0.711357607909332, 0.465015479876161, 0.45807...",None,0.700951,...,"(512, 512)",0.1,0.0001,1234,focal,2,True,5,64,768


In [22]:
best_ffn_mean_history_kfold["f1"].mean()

0.5258547665667903

In [23]:
best_ffn_mean_history_kfold["precision"].mean()

0.5186276424983707

In [24]:
best_ffn_mean_history_kfold["recall"].mean()

0.5408433751315166

In [25]:
np.stack(best_ffn_mean_history_kfold["f1_scores"]).mean(axis=0)

array([0.73682096, 0.46582395, 0.3749194 ])

In [26]:
np.stack(best_ffn_mean_history_kfold["precision_scores"]).mean(axis=0)

array([0.76500032, 0.46897879, 0.32190382])

In [27]:
np.stack(best_ffn_mean_history_kfold["recall_scores"]).mean(axis=0)

array([0.71068242, 0.46278638, 0.44906133])